# Applying the Skinning Animation using Euclidean representation forms

In the following, we will apply the skinning animation using the Euclidean representation forms. The model data is loaded from a dae file
using the pyassimp library. The skinning animation is applied using the linear blend skinning (LBS) method. The model data is alse stored in pickle files for later use.

In [ ]:

from Elements.definitions import MODEL_DIR, PICKLES_DIR
from os import path
from Elements.features.SkinnedMesh.gate_module import *
from pyassimp import load
import time 
import pickle

# Loading the model 

In [ ]:

object_path = MODEL_DIR / "astroBoy_walk.dae"
mesh_id = 3 # our model contains many meshes, we choose to visualize the 4th one

object_path_str = str(object_path) # convert possix.path to string so that pyassimp's load can read it
object = load(object_path_str)

mesh = object.meshes[mesh_id]
v = mesh.vertices
f = mesh.faces
b = mesh.bones
vw = vertex_weight(len(v))
vw.populate(b)

BB = [b[i].offsetmatrix for i in range(len(b))]


# Visualizing the T-pose of the model

In [ ]:
p = mp.plot(v, f, v[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  

## Visualizing the rest pose of the model

In [ ]:
# =================================================
# APPLYING EXTRA TRANSFORMATIONS IN JOINTS
# =================================================

transform = False  # True for applying extra transformations in joints
M = initialize_M(b) # initialize M with identity matrices
MM0 = read_tree(object,mesh_id,M,transform)

newv = np.zeros([(len(v)),3])
start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM0[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

# =================================================
# USING MESH PLOT OUTSIDE JUPYTER
# mp.offline()

print("TIME : ", end-start)
print("TRANSFORMATION = ", transform)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
# p.save("skinning.html")



## Saving the data that corresponds to the model and the pose, we may replicate the final result without loading the code again

In [ ]:
wanna_save = True


path_to_save = PICKLES_DIR/"astroboy_walk"

if wanna_save:
    with open(path_to_save/'vertices.pkl', 'wb') as file: pickle.dump(v, file)
    with open(path_to_save/'faces.pkl', 'wb') as file: pickle.dump(f, file)
    with open(path_to_save/'vw_id.pkl', 'wb') as file: pickle.dump(vw.id, file)
    with open(path_to_save/'vw_weight.pkl', 'wb') as file: pickle.dump(vw.weight, file)
    with open(path_to_save/'MM0.pkl', 'wb') as file: pickle.dump(MM0, file)
    with open(path_to_save/'BB.pkl', 'wb') as file: pickle.dump(BB, file)
    with open(path_to_save/'newv.pkl', 'wb') as file: pickle.dump(newv, file)

# should we use wb or w in the code above?

# Generating a different pose by changing the joint transformations

In [ ]:
# =================================================
# APPLYING EXTRA TRANSFORMATIONS IN JOINTS
# =================================================

transform = True  
M = initialize_M(b)


# the code below is used to apply extra transformations in joints
M[1][0:3,0:3] = eulerAnglesToRotationMatrix([0.3,0.3,0.4]) # applies a rotation at joint 1, in the x,y,z axis respectively (Euler Angles)
# M[1][0:3,3] = [5,5,5] # applies a translation at joint 1, by by the vector [5,5,5]
# M[1] = np.dot(np.diag([2,2,2,1]),M[1]) # applies a scaling at joint 1, by the vector [2,2,2]
MM1 = read_tree(object,mesh_id,M,transform)

path_to_save = PICKLES_DIR/"astroboy_walk"
with open(path_to_save/'MM1.pkl', 'wb') as file: pickle.dump(MM1, file)
# you can find these matrices in the pickles folder in MM1.pkl

# =================================================
# Applying the skinning
# =================================================

newv = np.zeros([(len(v)),3])
start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM1[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

print("TIME : ", end-start)
print("TRANSFORMATION = ", transform)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
# p.save("skinning.html")



# LOADING MODEL FROM PICKLE FILES

In case pyassimp loading fails, we can load the model from pickle files, as shown below.

In [ ]:

wanna_load = True

path_to_load = PICKLES_DIR/"astroboy_walk"

if wanna_load:
    with open(path_to_load/'vertices.pkl', 'rb') as file: v=pickle.load(file)
    with open(path_to_load/'faces.pkl', 'rb') as file: f=pickle.load(file)
    vw = vertex_weight(len(v))
    with open(path_to_load/'vw_id.pkl', 'rb') as file: vw.id = pickle.load(file)
    with open(path_to_load/'vw_weight.pkl', 'rb') as file: vw.weight = pickle.load(file)
    with open(path_to_load/'MM0.pkl', 'rb') as file: MM0 = pickle.load(file)
    with open(path_to_load/'MM1.pkl', 'rb') as file: MM1 = pickle.load(file)
    with open(path_to_load/'BB.pkl', 'rb') as file: BB = pickle.load(file)
    

MM = MM0 
# MM = MM1

# =================================================
# Applying the skinning
# =================================================
newv = np.zeros([(len(v)),3])
start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM1[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

# =================================================
# PLOTTING
# =================================================

print("TIME : ", end-start)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
# p.save("skinning.html")



# Simply printing the model 

In [ ]:
path_to_load = PICKLES_DIR/"astroboy_walk"
with open(path_to_load/'faces.pkl', 'rb') as file: f=pickle.load(file)
with open(path_to_load/'newv.pkl', 'rb') as file: newv = pickle.load(file)

p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  

# Generate more MM matrices

Note that the pose of the model is not changed, only the rest pose is changed. Therefore multiple poses can be generated from the same rest pose, using the following code.

In [ ]:
M = initialize_M(b)

M[1][0:3,0:3] = eulerAnglesToRotationMatrix([0.3,0.3,-0.4])
M[2][0:3,0:3] = eulerAnglesToRotationMatrix([0.3,-0.3,0.4])
# M[3][0:3,0:3] = eulerAnglesToRotationMatrix([0.3,0.3,0.4])
M[1][0:3,3] = [0,1,0]
M[1] = np.dot(np.diag([2,2,2,1]),M[1])

MM2 = read_tree(object,mesh_id,M,transform)
path_to_save = PICKLES_DIR/"astroboy_walk"
with open(path_to_save/'MM2.pkl', 'wb') as file: pickle.dump(MM2, file)

In [ ]:
MM = MM2
newv = np.zeros([(len(v)),3])
start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

# =================================================
# PLOTTING
# =================================================

print("TIME : ", end-start)
print("TRANSFORMATION = ", transform)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  